## As I need to generate the input information for the patent outputs. I have used "microsoft/Phi-3-mini-128k-instruct" for generating the inputs by using the columns title, abstract, description and claims. By prompt engineering I was able to create some inputs with column names as  related_art,  problem_statement , field , drawings , additional_details


In [1]:
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from tqdm import tqdm  # Import tqdm for the progress bar

# Load the microsoft/phi-3-mini-4k-instruct model and tokenizer from Hugging Face
model_name = "microsoft/Phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set pad_token to eos_token (common practice for causal models)
tokenizer.pad_token = tokenizer.eos_token

# Ensure you are using a GPU if available for faster inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load the CSV file with patent outputs (targets)
file_path = r"D:/Topcoder/patent_documentation/random_1000_abstractnotnull_1.csv"
df = pd.read_csv(file_path)

# Function to truncate text to approximately 100 tokens
def truncate_to_100_tokens(text):
    tokens = tokenizer.tokenize(text)
    truncated_text = tokenizer.convert_tokens_to_string(tokens[:100])
    return truncated_text

# Function to generate related art
def generate_related_art(row):
    prompt = f"""
    Based on the following patent information:
    - Abstract: {row['abstract']}
    - Description: {row['description']}
    
    Generate a detailed description of the related art, including the closest existing technology relevant to this patent.
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024, padding=True).to(device)
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=100,  
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    related_art = truncate_to_100_tokens(generated_text)

    if not related_art.strip():
        related_art = truncate_to_100_tokens(row['abstract']) or truncate_to_100_tokens(row['description'])

    return related_art

# Function to generate a clear problem statement
def generate_problem_statement(row):
    prompt = f"""
    Based on the following patent information:
    - Description: {row['description']}
    - Claims: {row['claims']}

    What is the main problem that this invention is solving in comparison to the related technology?
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024, padding=True).to(device)
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return truncate_to_100_tokens(generated_text)

# Function to define the technical field of the invention
def generate_field(row):
    prompt = f"""
    Based on the following patent information:
    - Title: {row['title']}
    - Description: {row['description']}

    Define the technical field of the invention.
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024, padding=True).to(device)
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=50,  
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return truncate_to_100_tokens(generated_text)

# Function to describe the drawings
def generate_drawings(row):
    prompt = f"""
    Based on the following patent information:
    - Claims: {row['claims']}
    - Description: {row['description']}

    Provide a brief description of the drawings or figures that might be included.
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024, padding=True).to(device)
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=50,  
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return truncate_to_100_tokens(generated_text)

# Function to generate concise additional details (2-3 sentences)
def generate_concise_additional_details(row):
    prompt = f"""
    Based on the following patent information:
    - Description: {row['description']}
    
    Provide 2-3 sentences of additional details that could be relevant for the patent application.
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024, padding=True).to(device)

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=50,  
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    additional_details = ' '.join(generated_text.split('. ')[:3]) + '.'

    return additional_details

# Function to prompt microsoft/phi-3-mini-128k-instruct to generate inputs
def generate_inputs(row):
    related_art = generate_related_art(row)
    problem_statement = generate_problem_statement(row)
    field = generate_field(row)
    drawings = generate_drawings(row)
    additional_details = generate_concise_additional_details(row)

    return related_art, problem_statement, field, drawings, additional_details

# Timer start
start_time = time.time()

# Create new columns in the DataFrame for the generated inputs with a progress bar
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Generating inputs"):
    related_art, problem_statement, field, drawings, additional_details = generate_inputs(row)
    df.at[index, 'related_art'] = related_art
    df.at[index, 'problem_statement'] = problem_statement
    df.at[index, 'field'] = field
    df.at[index, 'drawings'] = drawings
    df.at[index, 'additional_details'] = additional_details

# Save the updated DataFrame to a new CSV file
output_file = r"D:/Topcoder/patent_documentation/patent_data_with_inputs_phi3_mini_128k_fixed.csv"
df.to_csv(output_file, index=False)

# Timer end
end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time} seconds")
print(f"Generated inputs saved to {output_file}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating inputs: 100%|█████████████████████████████████████████████████████████| 1000/1000 [5:35:59<00:00, 20.16s/it]


Execution time: 20160.887031555176 seconds
Generated inputs saved to D:/Topcoder/patent_documentation/patent_data_with_inputs_phi3_mini_128k_fixed.csv


In [12]:
df['problem_statement'].values[:2]

array(['\n    Based on the following patent information:\n    - Description: CROSS-REFERENCE TO RELATED APPLICATION \n     This application claims benefit of priority under 35 USC 119 based on Japanese Patent Application No. 2018-150311 filed on Aug. 9, 2018, the entire contents of which are incorporated by reference herein. \n',
       '\n    Based on the following patent information:\n    - Description: TECHNICAL FIELD \n     The present application is filed claiming the priority of the Japanese Patent Application No. 2011-193237, the entire contents of which are herein incorporated by references. \n     The present invention relates to fluorescent molecular probes for implementing fluorescent detection (visualization) of tumors'],
      dtype=object)

In [7]:
df['field'].values[:2]

array(['\n    Based on the following patent information:\n    - Title: Dummy element and method of examining defect of resistive element\n    - Description: CROSS-REFERENCE TO RELATED APPLICATION \n     This application claims benefit of priority under 35 USC 119 based on Japanese Patent Application No. 2018-150311 filed on Aug. 9, 20',
       '\n    Based on the following patent information:\n    - Title: Polymer-type fluorescent molecule probe\n    - Description: TECHNICAL FIELD \n     The present application is filed claiming the priority of the Japanese Patent Application No. 2011-193237, the entire contents of which are herein incorporated by references. \n     The present invention relates to fluores'],
      dtype=object)

In [3]:
df['drawings'].values[0]

'\n    Based on the following patent information:\n    - Claims: What is claimed is: \n     \n       1. A dummy element for simulating a defective state of a resistive element of a target to be examined, the dummy element comprising:\n a semiconductor substrate; \n a lower insulating film deposited on the semiconductor substrate; \n a first resistive layer deposited on the lower insulating film'

In [8]:
df['drawings'].values[:2]

array(['\n    Based on the following patent information:\n    - Claims: What is claimed is: \n     \n       1. A dummy element for simulating a defective state of a resistive element of a target to be examined, the dummy element comprising:\n a semiconductor substrate; \n a lower insulating film deposited on the semiconductor substrate; \n a first resistive layer deposited on the lower insulating film',
       '\n    Based on the following patent information:\n    - Claims: The invention claimed is: \n     \n       1. A complex comprising a fluorescent molecule and a biocompatible macromolecule,\n wherein the fluorescent molecule is chlorophyll, \n the biocompatible macromolecule is selected from styrene-maleic acid copolymers, styrene-maleic acid copolym'],
      dtype=object)

In [9]:
df['additional_details'].values[:2]

array(['\n    Based on the following patent information:\n    - Description: CROSS-REFERENCE TO RELATED APPLICATION \n     This application claims benefit of priority under 35 USC 119 based on Japanese Patent Application No 2018-150311 filed on Aug 9, 2018, the entire contents of which are incorporated by reference herein.',
       '\n    Based on the following patent information:\n    - Description: TECHNICAL FIELD \n     The present application is filed claiming the priority of the Japanese Patent Application No 2011-193237, the entire contents of which are herein incorporated by references \n     The present invention relates to fluorescent molecular probes for implementing fluorescent detection (visualization) of tumors or, in addition to the fluorescent detection, photodynamic treatment, for instance, by using fluorescence endoscope or fluorescent laparoscope.'],
      dtype=object)

In [10]:
df['related_art'].values[:2]

array(['\n    Based on the following patent information:\n    - Abstract: A dummy element includes: a semiconductor substrate; a lower insulating film deposited on the semiconductor substrate; a first resistive layer deposited on the lower insulating film; an interlayer insulating film covering the first resistive layer; a first pad-forming electrode deposited on the interlayer insulating film so as to be connected to the first resistive layer',
       '\n    Based on the following patent information:\n    - Abstract: The present invention provides a fluorescent molecular probe for efficient fluorescent detection (visualization) of tumors or for implementing fluorescent detection and photodynamic treatment, more specifically, the present invention provides a macromolecular fluorescent molecular probe for fluorescent detection of tumor, comprising a complex comprising a fluorescent molecule and a bio'],
      dtype=object)

In [11]:
df.head()

,publication_number,title,abstract,claims,description,related_art,problem_statement,field,drawings,additional_details
0,US-11114351-B2,Dummy element and method of examining defect o...,A dummy element includes: a semiconductor subs...,What is claimed is: \n \n 1. A dummy...,CROSS-REFERENCE TO RELATED APPLICATION \n ...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...
1,US-10946109-B2,Polymer-type fluorescent molecule probe,The present invention provides a fluorescent m...,The invention claimed is: \n \n 1. A...,TECHNICAL FIELD \n The present application...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...
2,US-11112260-B2,Geospatial navigation methods and systems for ...,An exemplary geospatial navigation system defi...,What is claimed is: \n \n 1. A metho...,BACKGROUND INFORMATION \n Use of mobile na...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...
3,US-10940384-B2,Inciting user action for motion sensor calibra...,"In a method of motion sensor calibration, a mo...",What is claimed is: \n \n 1. A metho...,CROSS-REFERENCE TO RELATED APPLICATION—PROVISI...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...
4,US-2021298305-A1,Use of a difluoro-(2-hydroxypropyl)pyridine co...,The present disclosure is related to the field...,What is claimed is: \n \n 1 . A me...,CROSS-REFERENCE TO RELATED APPLICATION(S) \n ...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...


# Clean the file - we need to remove unnecessary information

In [13]:
to_clean_df = df.copy()
to_clean_df.head()

,publication_number,title,abstract,claims,description,related_art,problem_statement,field,drawings,additional_details
0,US-11114351-B2,Dummy element and method of examining defect o...,A dummy element includes: a semiconductor subs...,What is claimed is: \n \n 1. A dummy...,CROSS-REFERENCE TO RELATED APPLICATION \n ...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...
1,US-10946109-B2,Polymer-type fluorescent molecule probe,The present invention provides a fluorescent m...,The invention claimed is: \n \n 1. A...,TECHNICAL FIELD \n The present application...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...
2,US-11112260-B2,Geospatial navigation methods and systems for ...,An exemplary geospatial navigation system defi...,What is claimed is: \n \n 1. A metho...,BACKGROUND INFORMATION \n Use of mobile na...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...
3,US-10940384-B2,Inciting user action for motion sensor calibra...,"In a method of motion sensor calibration, a mo...",What is claimed is: \n \n 1. A metho...,CROSS-REFERENCE TO RELATED APPLICATION—PROVISI...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...
4,US-2021298305-A1,Use of a difluoro-(2-hydroxypropyl)pyridine co...,The present disclosure is related to the field...,What is claimed is: \n \n 1 . A me...,CROSS-REFERENCE TO RELATED APPLICATION(S) \n ...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...


In [14]:
to_clean_df.columns

Index(['publication_number', 'title', 'abstract', 'claims', 'description',
       'related_art', 'problem_statement', 'field', 'drawings',
       'additional_details'],
      dtype='object')

In [34]:
import pandas as pd

# Load the existing CSV file with the columns that need cleaning
file_path = r"D:/Topcoder/patent_documentation/patent_data_with_inputs_phi3_mini_128k_fixed.csv"
df = pd.read_csv(file_path)

# Define a function to clean specific patterns from a column's text
def clean_column(text, patterns_to_remove):
    if isinstance(text, str):
        for pattern in patterns_to_remove:
            text = text.replace(pattern, "")
    return text.strip()

# Patterns to remove from each column
patterns_related_art = ["\n    Based on the following patent information:\n    - Abstract:"]
patterns_problem_statement = ["\n    Based on the following patent information:\n    - Description:"]
patterns_field = ["\n    Based on the following patent information:\n    - Title:", "\n    - Description:"]
patterns_drawings = ["\n    Based on the following patent information:\n    - Claims:"]
patterns_additional_details = ["Based on the following patent information:\n    - Description:"]

# Apply cleaning to each relevant column and create new cleaned columns
df['cleaned_related_art'] = df['related_art'].apply(lambda x: clean_column(x, patterns_related_art))
df['cleaned_problem_statement'] = df['problem_statement'].apply(lambda x: clean_column(x, patterns_problem_statement))
df['cleaned_field'] = df['field'].apply(lambda x: clean_column(x, patterns_field))
df['cleaned_drawings'] = df['drawings'].apply(lambda x: clean_column(x, patterns_drawings))
df['cleaned_additional_details'] = df['additional_details'].apply(lambda x: clean_column(x, patterns_additional_details))

# Save the updated DataFrame to a new CSV file
output_file = r"D:/Topcoder/patent_documentation/patent_data_with_cleaned_inputs_phi3_mini.csv"
df.to_csv(output_file, index=False)

print(f"Cleaned data saved to {output_file}")


Cleaned data saved to D:/Topcoder/patent_documentation/patent_data_with_cleaned_inputs_phi3_mini.csv


In [35]:
df

,publication_number,title,abstract,claims,description,related_art,problem_statement,field,drawings,additional_details,cleaned_related_art,cleaned_problem_statement,cleaned_field,cleaned_drawings,cleaned_additional_details
0,US-11114351-B2,Dummy element and method of examining defect o...,A dummy element includes: a semiconductor subs...,What is claimed is: \n \n 1. A dummy...,CROSS-REFERENCE TO RELATED APPLICATION \n ...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,A dummy element includes: a semiconductor subs...,CROSS-REFERENCE TO RELATED APPLICATION \n ...,Dummy element and method of examining defect o...,What is claimed is: \n \n 1. A dummy...,CROSS-REFERENCE TO RELATED APPLICATION \n ...
1,US-10946109-B2,Polymer-type fluorescent molecule probe,The present invention provides a fluorescent m...,The invention claimed is: \n \n 1. A...,TECHNICAL FIELD \n The present application...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,The present invention provides a fluorescent m...,TECHNICAL FIELD \n The present application...,Polymer-type fluorescent molecule probe TECHNI...,The invention claimed is: \n \n 1. A...,TECHNICAL FIELD \n The present application...
2,US-11112260-B2,Geospatial navigation methods and systems for ...,An exemplary geospatial navigation system defi...,What is claimed is: \n \n 1. A metho...,BACKGROUND INFORMATION \n Use of mobile na...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,An exemplary geospatial navigation system defi...,BACKGROUND INFORMATION \n Use of mobile na...,Geospatial navigation methods and systems for ...,What is claimed is: \n \n 1. A metho...,BACKGROUND INFORMATION \n Use of mobile na...
3,US-10940384-B2,Inciting user action for motion sensor calibra...,"In a method of motion sensor calibration, a mo...",What is claimed is: \n \n 1. A metho...,CROSS-REFERENCE TO RELATED APPLICATION—PROVISI...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,"In a method of motion sensor calibration, a mo...",CROSS-REFERENCE TO RELATED APPLICATION—PROVISI...,Inciting user action for motion sensor calibra...,What is claimed is: \n \n 1. A metho...,CROSS-REFERENCE TO RELATED APPLICATION—PROVISI...
4,US-2021298305-A1,Use of a difluoro-(2-hydroxypropyl)pyridine co...,The present disclosure is related to the field...,What is claimed is: \n \n 1 . A me...,CROSS-REFERENCE TO RELATED APPLICATION(S) \n ...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,The present disclosure is related to the field...,CROSS-REFERENCE TO RELATED APPLICATION(S) \n ...,Use of a difluoro-(2-hydroxypropyl)pyridine co...,What is claimed is: \n \n 1. A met...,CROSS-REFERENCE TO RELATED APPLICATION(S) \n ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,US-11112786-B2,Remote trailer maneuver-assist,A method includes detecting a continuous rotar...,What is claimed is: \n \n 1. A metho...,BACKGROUND \n A vehicle may be equipped wi...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the following patent informatio...,\n Based on the followin

In [36]:
df.columns

Index(['publication_number', 'title', 'abstract', 'claims', 'description',
       'related_art', 'problem_statement', 'field', 'drawings',
       'additional_details', 'cleaned_related_art',
       'cleaned_problem_statement', 'cleaned_field', 'cleaned_drawings',
       'cleaned_additional_details'],
      dtype='object')

In [37]:
df[['cleaned_related_art',
       'cleaned_problem_statement', 'cleaned_field', 'cleaned_drawings',
       'cleaned_additional_details']]

,cleaned_related_art,cleaned_problem_statement,cleaned_field,cleaned_drawings,cleaned_additional_details
0,A dummy element includes: a semiconductor subs...,CROSS-REFERENCE TO RELATED APPLICATION \n ...,Dummy element and method of examining defect o...,What is claimed is: \n \n 1. A dummy...,CROSS-REFERENCE TO RELATED APPLICATION \n ...
1,The present invention provides a fluorescent m...,TECHNICAL FIELD \n The present application...,Polymer-type fluorescent molecule probe TECHNI...,The invention claimed is: \n \n 1. A...,TECHNICAL FIELD \n The present application...
2,An exemplary geospatial navigation system defi...,BACKGROUND INFORMATION \n Use of mobile na...,Geospatial navigation methods and systems for ...,What is claimed is: \n \n 1. A metho...,BACKGROUND INFORMATION \n Use of mobile na...
3,"In a method of motion sensor calibration, a mo...",CROSS-REFERENCE TO RELATED APPLICATION—PROVISI...,Inciting user action for motion sensor calibra...,What is claimed is: \n \n 1. A metho...,CROSS-REFERENCE TO RELATED APPLICATION—PROVISI...
4,The present disclosure is related to the field...,CROSS-REFERENCE TO RELATED APPLICATION(S) \n ...,Use of a difluoro-(2-hydroxypropyl)pyridine co...,What is claimed is: \n \n 1. A met...,CROSS-REFERENCE TO RELATED APPLICATION(S) \n ...
...,...,...,...,...,...
995,A method includes detecting a continuous rotar...,BACKGROUND \n A vehicle may be equipped wi...,Remote trailer maneuver-assist BACKGROUND \n ...,What is claimed is: \n \n 1. A metho...,BACKGROUND \n A vehicle may be equipped wi...
996,A flame retardant resin composition containing...,TECHNICAL FIELD \n One or more embodiments...,Flame retardant resin composition and metal ca...,The invention claimed is: \n \n 1. A...,TECHNICAL FIELD \n One or more embodiments...
997,Various methods and systems are provided for g...,FIELD \n Embodiments of the subject matter...,Methods and systems for protocol management FI...,"1. A system, comprising:\n a display; and a ...",FIELD \n Embodiments of the subject matter...
998,A terminal device may include: a communication...,TECHNICAL FIELD \n Embodiments disclosed r...,Device and method for time notification for up...,What is claimed is: \n \n 1. A termi...,TECHNICAL FIELD \n Embodiments disclosed r...


In [38]:
df['cleaned_additional_details'][:5].values

array(['CROSS-REFERENCE TO RELATED APPLICATION \n     This application claims benefit of priority under 35 USC 119 based on Japanese Patent Application No 2018-150311 filed on Aug 9, 2018, the entire contents of which are incorporated by reference herein.',
       'TECHNICAL FIELD \n     The present application is filed claiming the priority of the Japanese Patent Application No 2011-193237, the entire contents of which are herein incorporated by references \n     The present invention relates to fluorescent molecular probes for implementing fluorescent detection (visualization) of tumors or, in addition to the fluorescent detection, photodynamic treatment, for instance, by using fluorescence endoscope or fluorescent laparoscope.',
       'BACKGROUND INFORMATION \n     Use of mobile navigation devices has proliferated in recent years For example, along with dedicated navigation devices (e.g., standalone navigation devices, dedicated navigation devices built into vehicles, etc.), genera